# Quantum state reconstruction of the three-qubit constituents

In this notebook we use maximum-likelihood algorithm to reconstruct the three-qubit constituent states.
We will then resample the tomograms, assuming multinomial distribution and reconstruct each of them, saving the density matrices into h5 file.

These reconstructed density matrices will be subsequently analyzed for biseparability, fidelity, and other figures of merit.

In [1]:
import itertools
import numpy as np
import MaxLik as ml
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm

MC_SAMPLES = 100
ML_ITERS = 1000
ML_THR = 1e-12
SHOTS = 200

LO = np.array([[1],[0]])
HI = np.array([[0],[1]])
HLO = (LO+HI)*(2**-.5)
HHI = (LO-HI)*(2**-.5)
CLO = (LO+1j*HI)*(2**-.5)
CHI = (LO-1j*HI)*(2**-.5)


def kron(*arrays) -> np.ndarray:
    """
    Multiple Kronecker (tensor) product.
    Multiplication is performed from left.    
    """
    E = np.eye(1, dtype=complex)
    for M in arrays:
        E = np.kron(E,M)
    return E


eigenbase_dict = {
    'I' : ((LO, HI), (1,1)),
    'X' : ((HLO, HHI), (1,-1)), #no sign flip
    'Y' : ((CLO, CHI),(1,-1)),
    'Z' : ((LO, HI), (1,-1)),
}

def base_string_to_proj(string : str) -> list:
    """
    Input measurement string and get projection operators corresponding to that string, 
    all combinations that can happen.    
    """
    eigenvects = [eigenbase_dict.get(b)[0] for b in string]
    proj_kets = [kron(*vecs) for i, vecs in enumerate(itertools.product(*eigenvects))]
    return proj_kets



MaxLik: Numba Allowed: True => use cycle-based K-vector construction


In [2]:
#Load tomographic data
with h5py.File('data/three_qubit_tomograms.h5') as h5:
    data_024 = np.array(h5['ions_024'])
    data_531 = np.array(h5['ions_531'])
    meas_order = list(h5['ions_024'].attrs['tomo_order'])
    state_order = list(h5['ions_024'].attrs['state_order'])

meas_eigenkets = np.array([base_string_to_proj(mstr) for mstr in meas_order])
meas_eigenkets = meas_eigenkets.reshape((27*8, 8, 1))

In [3]:
#Reconstruct experimentally observed data
rho_seed_024 = np.array([ml.Reconstruct(tomogram.ravel(), meas_eigenkets, ML_ITERS, ML_THR) for tomogram in data_024])
rho_seed_531 = np.array([ml.Reconstruct(tomogram.ravel(), meas_eigenkets, ML_ITERS, ML_THR) for tomogram in data_531])

In [4]:
#Do Monte-Carlo resampling of tomograms and reconstruct them

#fix seed number for reproducibility
rng = np.random.default_rng(seed=24092025)

def multiaxis_multinomial(arr, rng, samples, shots):
    """
    Support for multiple axes in rng.multinomial for older numpy version.
    """
    shape = arr.shape
    new_shape = (-1, shape[-1])
    temp_arr = np.array([rng.multinomial(shots, line, size=samples) for line in arr.reshape(new_shape)])
    return temp_arr.swapaxes(0,1).reshape((samples, *shape))

labels = ('024', '531')

with h5py.File('data/reconstructed_three_qubit_states.h5', 'w') as h5:
    dset_orig_1 = h5.create_dataset(f'rho_024', data=rho_seed_024)
    dset_orig_2 = h5.create_dataset(f'rho_531', data=rho_seed_531)
    h5.flush()

    for label, dataset in zip(labels, (data_024, data_531)):    
        print(f'Processing dataset: {label}')
        _dset = h5.create_dataset(f'rhos_{label}', shape=(MC_SAMPLES, 10, 8, 8), dtype=complex)
        #in modern numpy versions just use
        #mc_tomograms = rng.multinomial(SHOTS, dataset, size=(MC_SAMPLES,10,27)).reshape((MC_SAMPLES, 10, -1))
        sampled_tomograms = multiaxis_multinomial(dataset, rng, MC_SAMPLES, SHOTS)
        for i, _sample in tqdm(enumerate(sampled_tomograms), total=MC_SAMPLES):
            for j, _tomo in enumerate(_sample):
                _rho = ml.Reconstruct(_tomo.ravel(), meas_eigenkets, ML_ITERS, ML_THR)
                _dset[i,j] = _rho
        h5.flush()
print("Done")

Processing dataset: 024


100%|██████████| 100/100 [02:11<00:00,  1.31s/it]


Processing dataset: 531


100%|██████████| 100/100 [02:21<00:00,  1.41s/it]

Done
